# Welcom to EdgeLab for Google Colab Training Example

In this tutorial, we will demonstrate how to use EdgeLab and Colab to train a PFLD model for meter reading.

For more details of PFLD model or application scenarios, please refer to [EdgeLab Documentation](https://seeed-studio.github.io/EdgeLab/).

## Setup EdgeLab

In [ ]:
!git clone https://github.com/Unbinilium/EdgeLab.git -b 'lab2.0' # currently we're using experimental 2.0 version branch

In [ ]:
%cd /content/EdgeLab

In [ ]:
%env PYTHON_VERSION=python3.8

In [ ]:
!sudo apt-get update
!sudo apt-get install ${PYTHON_VERSION}-dev python3-pip -y

In [ ]:
!chmod +x scripts/setup_colab.sh
!scripts/setup_colab.sh $(which ${PYTHON_VERSION})

## Download Custom Datasets

In [ ]:
!wget https://files.seeedstudio.com/wiki/Edgelab/meter.zip -P datasets/
!unzip datasets/meter.zip -d datasets/

## Train the Model

### Examine the PFLD config file

The config we use for train is located at `configs/pfld/pfld_mv2n_112.py`.

### Train the model using default config

Then, train PFLD model using default config, note here we need to override the config by 2 parameters:

- `data_root` - the datasets path, which located at path `datasets/meter`

- `epochs`- the train epochs, we use `50` to reduce the training time

In [ ]:
!${PYTHON_VERSION} tools/train.py pose configs/pfld/pfld_mv2n_112.py --cfg-options epochs=50 data_root='datasets/meter'

## Export PyTorch Model to TFLite

### Get latest weights

The training output of the default confing is located at path `work_dirs/pfld_mv2n_112`, before exporting the trained PFLD model to a TFLite model, we're suppose to find the latest weights.

In [ ]:
import os
import glob

In [ ]:
folder_path = 'work_dirs/pfld_mv2n_112'

pth_files = glob.glob(os.path.join(folder_path, '*.pth'))
sorted_pths = sorted(pth_files, key=os.path.getmtime, reverse=True)
latest_pth = sorted_pths[0]

%env LATEST_PTH={latest_pth}

### Export the latest PFLD weights to TFLite model

Here we're using `tools/torch2tflite.py` to convert and export the latest weights to a TFLite model at a INT8 precision for inference.

In [ ]:
!${PYTHON_VERSION} tools/torch2tflite.py \
    configs/pfld/pfld_mv2n_112.py \
    --checkpoint ${LATEST_PTH} \
    --type int8